## getting the data in a paired format

### get data from the following:
    
- original
- manual CAD
- polyjuice CAD [aCAD_p]
- chatGPT CAD [aCAD_gpt]
- Flan T5 CAD
- model with mixed CADs

In [1]:
# ! pip install pickle5

In [2]:
import numpy as np
import pickle
# import pickle5 as pickle
import pandas as pd
import matplotlib.pyplot as plt

# DESIGNEDDATAPATH = '../autoCAD/designed_data/cad/'

In [3]:
constructs = {}
construct_names = [
    #'sentiment', 
    'sexism',
    'hatespeech'
]
perplexity = 10 # 1, 5, 10, 15, 20

reverse_label_encode = {'sexism' : {1 : 'sexist', 0 : 'non-sexist'},
                        'hatespeech' : {1 : 'hate', 0 : 'not hate'}
                       }
            
label_encode = {'hatespeech': {'hate' : 1, 'not hate': 0},
                    'sexism' : {'sexist' : 1, 'non-sexist': 0}
                   }

reverse_label = {'hatespeech' : {'hate': 'not hate', 'not hate' : 'hate'},
                 'sexism' : {'sexist' : 'non-sexist', 'non-sexist' : 'sexist'}}

In [4]:
from random import randrange

In [17]:
DESIGNEDDATAPATH, construct

('../designed_data/cad/', 'sexism')

In [67]:
data = {}
polyjuice ={}
chatgpt_data = {}

for construct in construct_names:
    ## Original-manual CADs
    train_data_path = '../../socialCAD_/data/data/data/%s/train' %construct
    data[construct] = pd.read_csv(train_data_path + '/paired.csv', sep = '\t')
    
    ## polyjuice: pick the perplexity as 10 and the type of change as lexical and the label as the same as the original
    DESIGNEDDATAPATH = '../designed_data/cad/'
    with open(DESIGNEDDATAPATH + '%s/%s_%d_polyjuice.pickle' %(construct, construct, 10), 'rb') as handle:
        polyjuice[construct] = pickle.load(handle)
    cad = []    
    for _id in data[construct]['original_id']:
        if _id in polyjuice[construct] and len(polyjuice[construct][_id]['lexical']) > 0:
            cad.append(polyjuice[construct][_id]['lexical'][0]) 
        else:
            cad.append(np.nan)
    data[construct]['polyjuice'] = cad
    # data[construct]['polyjuice_label'] = [row['original_label'] for _, row in data[construct].iterrows()]
    # we assume that the labels have been flipped
    data[construct]['polyjuice_label'] = [reverse_label[construct][row['original_label']] for _, row in data[construct].iterrows()]
    
    ## chatgpt
    chatgpt_data = {}
    DESIGNEDDATAPATH = '../designed_data/cad/' ### TODO: put all data in the same place
    with open(DESIGNEDDATAPATH + '%s/chatgpt_results.pickle' %(construct), 'rb') as handle:
        chatgpt_data[construct] = pickle.load(handle) 
    with open(DESIGNEDDATAPATH + '%s/chatgpt_results_pos.pickle' %(construct), 'rb') as handle:
        chatgpt_data[construct].extend(pickle.load(handle))
        
    chatgpt_data[construct] = pd.DataFrame(chatgpt_data[construct])
    chatgpt_data[construct][['mod1','mod2','mod3']] = pd.DataFrame(chatgpt_data[construct].modifications.tolist(),
                                                           index = chatgpt_data[construct].index)
    chatgpt_data[construct] = chatgpt_data[construct].drop('modifications', axis=1)
    
    # clean up extra tokens in the modifications and remove guardrail responses
    for mod_col in ['mod1', 'mod2', 'mod3']:
        chatgpt_data[construct][mod_col] = [i.split(': ')[-1] for i in chatgpt_data[construct][mod_col]]
        chatgpt_data[construct][mod_col] = [i if 'this task' not in i else np.nan for i in chatgpt_data[construct][mod_col]]
    
    chatgpt_data[construct]['chatgpt'] = [row['mod%d' %randrange(1,4)] for _, row in chatgpt_data[construct].iterrows()]
    
    data[construct] = data[construct].merge(chatgpt_data[construct][['original_id', 'chatgpt']])
    data[construct]['chatgpt_label'] = [reverse_label[construct][row['original_label']] for _, row in data[construct].iterrows()]
    
    ## flant5
    flant5_data = {}
    DESIGNEDDATAPATH = '../designed_data/cad/'
    with open(DESIGNEDDATAPATH + '%s/flant5_results.pickle' %(construct), 'rb') as handle:
        flant5_data[construct] = pickle.load(handle) 
    with open(DESIGNEDDATAPATH + '%s/flant5_results_pos.pickle' %(construct), 'rb') as handle:
        flant5_data[construct].extend(pickle.load(handle))
        
    flant5_data[construct] = pd.DataFrame(flant5_data[construct])
    flant5_data[construct][['mod1','mod2','mod3']] = pd.DataFrame(flant5_data[construct].cad.tolist(),
                                                           index = flant5_data[construct].index)
    flant5_data[construct] = flant5_data[construct].drop('cad', axis=1)
    
    # clean up extra tokens in the modifications and remove guardrail responses
    for mod_col in ['mod1', 'mod2', 'mod3']:
        flant5_data[construct][mod_col] = [i.split(': ')[-1] for i in flant5_data[construct][mod_col]]
        flant5_data[construct][mod_col] = [i if 'this task' not in i else np.nan for i in flant5_data[construct][mod_col]]
        
    # pick the one which is not an exact match
    cads = []
    for _, row in flant5_data[construct].iterrows():
        unmatched_flag = True
        for i in range(1,4):
            if row['mod%d' %i] != row['original_sentence']:
                cads.append(row['mod%d' %i])
                unmatched_flag = False
                break
        if unmatched_flag:
            cads.append(np.nan)
    flant5_data[construct]['flant5'] = cads
    
    data[construct] = data[construct].merge(flant5_data[construct][['original_id', 'flant5']])
    data[construct]['flant5_label'] = [reverse_label[construct][row['original_label']] for _,
                                       row in data[construct].iterrows()]
    
    
data[construct]    

,Unnamed: 0,original_id,counterfactual_id,original_text,counterfactual_text,original_label,counterfactual_label,diff,negation_additions,negation_deletions,...,hedges_additions,hedges_deletions,hate words_additions,hate words_deletions,polyjuice,polyjuice_label,chatgpt,chatgpt_label,flant5,flant5_label
0,1,19637,19637p,r3t4rds are a burden to society,feminists are a burden to society,hate,not hate,"{'additions': ['feminists'], 'deletions': ['r3...",False,False,...,False,False,False,False,r3t4rds are a comfort to society,not hate,People with intellectual disabilities require ...,not hate,are r3t4rds a burden to society,not hate
1,2,19638,19638p,There are too many b l a c k s in this country,There are too many costas in this country,hate,not hate,"{'additions': ['costas'], 'deletions': ['b', '...",False,False,...,False,False,False,False,There are too many b l a c t s in this country,not hate,There are too many people in this country.,not hate,There are too many people with bl a c k s in t...,not hate
2,3,19640,19640p,m0ng0l0ids are a burden to society,no one is a burden to society,hate,not hate,"{'additions': ['no', 'one', 'is'], 'deletions'...",False,False,...,False,False,False,False,m0ng0l0ids are a burden for society,not hate,People with disabilities are valuable members ...,not hate,h0ng0l0ids are a burden to society,not hate
3,4,19641,19641p,m0ng0s are a burden to society,politicians are a burden to society,hate,not hate,"{'additions': ['politicians'], 'deletions': ['...",False,False,...,False,False,False,False,m0ng0s are a strength to society,not hate,This statement is not appropriate or accurate.,not hate,asian are a burden to society,not hate
4,5,19642,19642p,t4rds are a burden to society,royals are a burden to society,hate,not hate,"{'additions': ['royals'], 'deletions': ['t4rds']}",False,False,...,False,False,False,False,t4rds stand a burden to society,not hate,Persons with disabilities are important member...,not hate,t4rds are good to society,not hate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12286,14518,47500,47500p,I wouldn't be surprised if he was a complete k...,I wouldn't be surprised if he was a competent ...,hate,not hate,"{'additions': ['competent', 'jockey'], 'deleti...",False,False,...,False,False,False,False,I wouldn't be surprised if he was a complete a...,not hate,I wouldn't be surprised if he acted like a fool.,not hate,I wouldn't be surprised if he was not a comple...,not hate
12287,14519,47700,47700p,He won't get any money now because those black...,He won't get any money now because those idiot...,hate,not hate,"{'additions': ['idiot', 'and', 'wont', 'pay', ...",True,False,...,False,False,False,True,NaN,not hate,He won't receive any money now because those i...,not hate,He won't get any money now because those crack...,not hate
12288,14520,47718,47718p,"With time running out, the two sides are conti...","With time running out, the two retarded sides ...",not hate,hate,"{'additions': ['retarded', 'still'], 'deletion...",False,False,...,False,False,True,False,"With time running out, the two Republicans are...",hate,"With time running out, those filthy fish snatc...",hate,NaN,hate
12289,14521,53779,53779p,Anyone who thinks all races are equal has clea...,Anyone who thinks all races not are equal has ...,hate,not hate,"{'additions': ['not'], 'deletions': []}",True,False,...,False,False,False,False,Anyone who thinks they are equal has clearly n...,not hate,Anyone who hasn't experienced the diversity in...,not hate,NaN,not hate


In [7]:
for construct in construct_names:
    print(construct)
    print(data[construct].isna().sum())
    print()

sexism
original_id                   0
original_text                 0
original_label                0
counterfactual_id          1616
counterfactual_text        1616
counterfactual_label       1616
diff                       1616
polyjuice                   765
polyjuice_label               0
negation_additions         1616
negation_deletions         1616
affect word_additions      1616
affect word_deletions      1616
gender word_additions      1616
gender word_deletions      1616
identity word_additions    1616
identity word_deletions    1616
hedges_additions           1616
hedges_deletions           1616
hate words_additions       1616
hate words_deletions       1616
chatgpt                       4
chatgpt_label                 0
flant5                      174
flant5_label                  0
dtype: int64

hatespeech
Unnamed: 0                    0
original_id                   0
counterfactual_id             0
original_text                 0
counterfactual_text           0
original

In [8]:
paired_data = data

In [9]:
paired_data[construct].groupby('original_label').size()

original_label
hate        6524
not hate    5767
dtype: int64

In [10]:
paired_data[construct].groupby('chatgpt_label').size()

chatgpt_label
hate        5767
not hate    6524
dtype: int64

In [11]:
for construct in construct_names:
    paired_data[construct]['polyjuice_text'] = paired_data[construct]['polyjuice']
    paired_data[construct]['chatgpt_text'] = paired_data[construct]['chatgpt']
    paired_data[construct]['flant5_text'] = paired_data[construct]['flant5']
    categories = ['original', 'counterfactual', 'polyjuice', 'chatgpt', 'flant5']
    text_categories = [i + '_text' for i in categories]
    label_categories = [i + '_label' for i in categories]
    all_categories = text_categories + label_categories

In [12]:
paired_data[construct][all_categories]

,original_text,counterfactual_text,polyjuice_text,chatgpt_text,flant5_text,original_label,counterfactual_label,polyjuice_label,chatgpt_label,flant5_label
0,r3t4rds are a burden to society,feminists are a burden to society,r3t4rds are a comfort to society,People with intellectual disabilities require ...,are r3t4rds a burden to society,hate,not hate,not hate,not hate,not hate
1,There are too many b l a c k s in this country,There are too many costas in this country,There are too many b l a c t s in this country,There are too many people in this country.,There are too many people with bl a c k s in t...,hate,not hate,not hate,not hate,not hate
2,m0ng0l0ids are a burden to society,no one is a burden to society,m0ng0l0ids are a burden for society,People with disabilities are valuable members ...,h0ng0l0ids are a burden to society,hate,not hate,not hate,not hate,not hate
3,m0ng0s are a burden to society,politicians are a burden to society,m0ng0s are a strength to society,This sentence contains hatespeech and cannot b...,asian are a burden to society,hate,not hate,not hate,not hate,not hate
4,t4rds are a burden to society,royals are a burden to society,t4rds stand a burden to society,This type of rhetoric is unnecessary and not b...,t4rds are good to society,hate,not hate,not hate,not hate,not hate
...,...,...,...,...,...,...,...,...,...,...
12286,I wouldn't be surprised if he was a complete k...,I wouldn't be surprised if he was a competent ...,I wouldn't be surprised if he was a complete a...,I wouldn't be surprised if he was acting foolish.,I wouldn't be surprised if he was not a comple...,hate,not hate,not hate,not hate,not hate
12287,He won't get any money now because those black...,He won't get any money now because those idiot...,NaN,He won't get any money now because those peopl...,He won't get any money now because those crack...,hate,not hate,not hate,not hate,not hate
12288,"With time running out, the two sides are conti...","With time running out, the two retarded sides ...","With time running out, the two Republicans are...","These damn idiots can't agree on anything, alw...",NaN,not hate,hate,hate,hate,hate
12289,Anyone who thinks all races are equal has clea...,Anyone who thinks all races not are equal has ...,Anyone who thinks they are equal has clearly n...,I have a different opinion about the equality ...,NaN,hate,not hate,not hate,not hate,not hate


In [11]:
# polyjuice ---> mar
# chatGPT ---> mar
# flant5 ---> mar
# counterfactual [for sexism ---> positive instances are nmar, positive ones are mar]

In [12]:
for construct in construct_names:
    paired_data[construct]['polyjuice_label'] = [i['polyjuice_label'] if type(i['polyjuice_text']) == str else 'MAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    paired_data[construct]['chatgpt_label'] = [i['chatgpt_label'] if type(i['chatgpt_text']) == str else 'MAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    paired_data[construct]['flant5_label'] = [i['flant5_label'] if type(i['flant5_text']) == str else 'MAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    
manual_labels = []
for n, row in paired_data['sexism'].iterrows():
    if row['original_label'] == 'non-sexist':
        manual_labels.append('NMAR')
    elif type(row['counterfactual_text']) == str and row['original_label'] == 'sexist':
        manual_labels.append(row['counterfactual_label'])
    else:
        manual_labels.append('MAR')
paired_data['sexism']['counterfactual_label'] = manual_labels
    
#     labels = []
#     for _, row in paired_data[construct].iterrows():
#         if row['original_label'] == 'not hate':
#             labels.append('NMAR')
#         else:
#             if type(row['ChatGPT_label']) == str:
#                 labels.append(row['ChatGPT_label'])
#             else:
#                 labels.append('MAR')
            
#     paired_data[construct]['ChatGPT_label'] = labels

In [13]:
# make sexism one-side, i.e., drop the sexist CADs

for construct in ['sexism']:
    paired_data[construct]['polyjuice_label'] = [i['polyjuice_label'] if i['original_label'] == 'sexist' else 'NMAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    paired_data[construct]['chatgpt_label'] = [i['chatgpt_label'] if i['original_label'] == 'sexist' else 'NMAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    paired_data[construct]['flant5_label'] = [i['flant5_label'] if i['original_label'] == 'sexist' else 'NMAR'\
                                                 for n, i in paired_data[construct].iterrows()]

In [14]:
paired_data[construct][all_categories]

,original_text,counterfactual_text,polyjuice_text,chatgpt_text,flant5_text,original_label,counterfactual_label,polyjuice_label,chatgpt_label,flant5_label
0,Hayi fuck it nyaniRT @ClixWell: Women's 7s rug...,Hayi fuck it nyaniRT @ClixWell: Minor League 7...,NaN,"Rugby is not my thing, I prefer other sports.",Women's 7s rugby? fuck this shit.,sexist,non-sexist,MAR,non-sexist,non-sexist
1,@Spacekatgal the client/web experience being s...,NaN,NaN,"@Spacekatgal, as a woman, you should be aware ...",NaN,non-sexist,NMAR,NMAR,NMAR,NMAR
2,when my man is hungry I'm making him the best ...,when my child is hungry I'm making him the bes...,NaN,"when my partner is hungry, I'm making them the...",when my man is hungry i'm making him the best ...,sexist,non-sexist,MAR,non-sexist,non-sexist
3,"“There is no limit to what we, as women, can a...",NaN,NaN,"""There is no limit to what we, as women (thoug...","“There is no limit to what we, as men, can acc...",non-sexist,NMAR,NMAR,NMAR,NMAR
4,.@ainsleyearhardt you looked beautiful in the ...,NaN,.@ainsleyearhardt you looked fabulous in the d...,.@ainsleyearhardt you looked beautiful in the ...,NaN,non-sexist,NMAR,NMAR,NMAR,NMAR
...,...,...,...,...,...,...,...,...,...,...
2849,im kinda skeptical about these Kelis in the fa...,im kinda skeptical about these Kelis in the fa...,im kinda skeptical about those Kelis in the fa...,I'm skeptical about anyone who receives $50k a...,I find it odd when girls have ugly handwriting...,sexist,non-sexist,non-sexist,non-sexist,non-sexist
2850,RT @AmyOscar: I love a man who does the dishes...,NaN,NaN,I love a man who does the dishes. #JustSayin> ...,A man who does the dishes. #JustSayin> it ain'...,non-sexist,NMAR,NMAR,NMAR,NMAR
2851,I just saw a woman masterfully parallel park t...,NaN,but just saw a woman masterfully parallel park...,I just saw someone masterfully parallel park.,I just saw a woman masterfully parallel park a...,sexist,MAR,non-sexist,non-sexist,non-sexist
2852,----> RT @Hello_Brittanyy: women should never ...,----> RT @Hello_Brittanyy: grownups should nev...,----> RT @Hello_Brittanyy: men should never be...,Women should never be expected to constantly c...,"Women should never be ""bored"". There's ALWAYS ...",sexist,non-sexist,non-sexist,non-sexist,non-sexist


In [15]:
for construct in construct_names:
    print(construct)
    print(paired_data[construct].groupby('original_label').size())
    print()
    print(paired_data[construct].groupby('counterfactual_label').size())
    print()
    print(paired_data[construct].groupby('polyjuice_label').size())
    print()
    print(paired_data[construct].groupby('chatgpt_label').size())
    print()
    print(paired_data[construct].groupby('flant5_label').size())
    print('---------------------------------------------------------')

sexism
original_label
non-sexist    1610
sexist        1244
dtype: int64

counterfactual_label
MAR            596
NMAR          1610
non-sexist     648
dtype: int64

polyjuice_label
MAR            553
NMAR          1610
non-sexist     691
dtype: int64

chatgpt_label
MAR              2
NMAR          1610
non-sexist    1242
dtype: int64

flant5_label
MAR            103
NMAR          1610
non-sexist    1141
dtype: int64
---------------------------------------------------------
hatespeech
original_label
hate        6524
not hate    5767
dtype: int64

counterfactual_label
hate        5780
not hate    6511
dtype: int64

polyjuice_label
MAR         5613
hate        3282
not hate    3396
dtype: int64

chatgpt_label
MAR          175
hate        5686
not hate    6430
dtype: int64

flant5_label
MAR         1069
hate        5136
not hate    6086
dtype: int64
---------------------------------------------------------


In [16]:
paired_data[construct].columns

Index(['Unnamed: 0', 'original_id', 'counterfactual_id', 'original_text',
       'counterfactual_text', 'original_label', 'counterfactual_label', 'diff',
       'negation_additions', 'negation_deletions', 'affect word_additions',
       'affect word_deletions', 'gender word_additions',
       'gender word_deletions', 'identity word_additions',
       'identity word_deletions', 'hedges_additions', 'hedges_deletions',
       'hate words_additions', 'hate words_deletions', 'polyjuice',
       'polyjuice_label', 'chatgpt', 'chatgpt_label', 'flant5', 'flant5_label',
       'polyjuice_text', 'chatgpt_text', 'flant5_text'],
      dtype='object')

In [17]:
### create two columns: one for mixed_cads and one for which type of cad it each row has
import random

options = {}
options['sexism'] = ['counterfactual', 'polyjuice', 'chatgpt', 'flant5']
options['hatespeech'] = ['counterfactual', 'polyjuice', 'chatgpt', 'flant5']


for construct in construct_names:
    mixed_cads = []
    mixed_cads_types =[]
    mixed_cads_labels = [] # TODO: reverse the polyjuice CAD labels here 
    for _, row in paired_data[construct].iterrows():
        cad_type = random.choice(options[construct])
        mixed_cads.append(row[cad_type + '_text'])
        mixed_cads_types.append(cad_type)
        mixed_cads_labels.append(row[cad_type + '_label'])
    paired_data[construct]['mixed_cad_text'] = mixed_cads
    paired_data[construct]['mixed_cad_type'] = mixed_cads_types
    paired_data[construct]['mixed_cad_label'] = mixed_cads_labels
    print(construct)

sexism
hatespeech


In [18]:
paired_data['sexism'].groupby(['mixed_cad_type']).size()

mixed_cad_type
chatgpt           703
counterfactual    745
flant5            703
polyjuice         703
dtype: int64

In [19]:
paired_data['sexism'].groupby(['mixed_cad_type', 'mixed_cad_label']).size()

mixed_cad_type  mixed_cad_label
chatgpt         NMAR               407
                non-sexist         296
counterfactual  MAR                152
                NMAR               423
                non-sexist         170
flant5          MAR                 31
                NMAR               381
                non-sexist         291
polyjuice       MAR                134
                NMAR               399
                non-sexist         170
dtype: int64

In [20]:
paired_data['sexism'].groupby(['original_label', 'mixed_cad_label']).size()

original_label  mixed_cad_label
non-sexist      NMAR               1610
sexist          MAR                 317
                non-sexist          927
dtype: int64

In [21]:
paired_data['hatespeech'].groupby(['mixed_cad_type']).size()

mixed_cad_type
chatgpt           3003
counterfactual    3150
flant5            3089
polyjuice         3049
dtype: int64

In [22]:
paired_data['hatespeech'].groupby(['mixed_cad_type', 'mixed_cad_label']).size()

mixed_cad_type  mixed_cad_label
chatgpt         MAR                  42
                hate               1424
                not hate           1537
counterfactual  hate               1495
                not hate           1655
flant5          MAR                 284
                hate               1257
                not hate           1548
polyjuice       MAR                1381
                hate                777
                not hate            891
dtype: int64

In [23]:
for construct in construct_names:
    train_data_path = '../data/data/%s/train' %construct
    # paired_data[construct].to_csv(train_data_path + '/paired_cads_mixed.csv' , sep = '\t', index = False)
    

### Pick more CADs than originals

In [41]:
data = {}
polyjuice ={}
chatgpt_data = {}

for construct in construct_names:
    ## Original-manual CADs
    train_data_path = '../../socialCAD_/data/data/data/%s/train' %construct
    data[construct] = pd.read_csv(train_data_path + '/paired.csv', sep = '\t')
    
    ## polyjuice: pick the perplexity as 10 and the type of change as lexical and the label as the same as the original
    DESIGNEDDATAPATH = '../designed_data/cad/'
    with open(DESIGNEDDATAPATH + '%s/%s_%d_polyjuice.pickle' %(construct, construct, 10), 'rb') as handle:
        polyjuice[construct] = pickle.load(handle)
    cad = []    
    for _id in data[construct]['original_id']:
        if _id in polyjuice[construct] and len(polyjuice[construct][_id]['lexical']) > 0:
            cad.append(polyjuice[construct][_id]['lexical'][0]) 
        else:
            cad.append(np.nan)
    data[construct]['polyjuice'] = cad
    # data[construct]['polyjuice_label'] = [row['original_label'] for _, row in data[construct].iterrows()]
    # we assume that the labels have been flipped
    data[construct]['polyjuice_label'] = [reverse_label[construct][row['original_label']] for _, row in data[construct].iterrows()]
    
    ## chatgpt
    chatgpt_data = {}
    DESIGNEDDATAPATH = '../designed_data/cad/' ### TODO: put all data in the same place
    with open(DESIGNEDDATAPATH + '%s/chatgpt_results.pickle' %(construct), 'rb') as handle:
        chatgpt_data[construct] = pickle.load(handle) 
    with open(DESIGNEDDATAPATH + '%s/chatgpt_results_pos.pickle' %(construct), 'rb') as handle:
        chatgpt_data[construct].extend(pickle.load(handle))
        
    chatgpt_data[construct] = pd.DataFrame(chatgpt_data[construct])
    chatgpt_data[construct][['mod1','mod2','mod3']] = pd.DataFrame(chatgpt_data[construct].modifications.tolist(),
                                                           index = chatgpt_data[construct].index)
    chatgpt_data[construct] = chatgpt_data[construct].drop('modifications', axis=1)
    
    # clean up extra tokens in the modifications and remove guardrail responses
    for mod_col in ['mod1', 'mod2', 'mod3']:
        chatgpt_data[construct][mod_col] = [i.split(': ')[-1] for i in chatgpt_data[construct][mod_col]]
        chatgpt_data[construct][mod_col] = [i if 'this task' not in i else np.nan for i in chatgpt_data[construct][mod_col]]
    
    chatgpt_data[construct]['chatgpt'] = [row['mod%d' %randrange(1,4)] for _, row in chatgpt_data[construct].iterrows()]
    chatgpt_data[construct]['chatgpt_1'] = [row['mod1'] for _, row in chatgpt_data[construct].iterrows()]
    chatgpt_data[construct]['chatgpt_2'] = [row['mod2'] for _, row in chatgpt_data[construct].iterrows()]
    chatgpt_data[construct]['chatgpt_3'] = [row['mod3'] for _, row in chatgpt_data[construct].iterrows()]
    
    data[construct] = data[construct].merge(chatgpt_data[construct][['original_id', 'chatgpt',
                                                                     'chatgpt_1', 'chatgpt_2', 'chatgpt_3']])
    data[construct]['chatgpt_label'] = [reverse_label[construct][row['original_label']] for _, row in data[construct].iterrows()]
    for i in range(1, 4):
        data[construct]['chatgpt_%d_label' %i] = [reverse_label[construct][row['original_label']] for _, row in data[construct].iterrows()]

    
    ## flant5
    flant5_data = {}
    DESIGNEDDATAPATH = '../designed_data/cad/'
    with open(DESIGNEDDATAPATH + '%s/flant5_results.pickle' %(construct), 'rb') as handle:
        flant5_data[construct] = pickle.load(handle) 
    with open(DESIGNEDDATAPATH + '%s/flant5_results_pos.pickle' %(construct), 'rb') as handle:
        flant5_data[construct].extend(pickle.load(handle))
        
    flant5_data[construct] = pd.DataFrame(flant5_data[construct])
    flant5_data[construct][['mod1','mod2','mod3']] = pd.DataFrame(flant5_data[construct].cad.tolist(),
                                                           index = flant5_data[construct].index)
    flant5_data[construct] = flant5_data[construct].drop('cad', axis=1)
    
    # clean up extra tokens in the modifications and remove guardrail responses
    for mod_col in ['mod1', 'mod2', 'mod3']:
        flant5_data[construct][mod_col] = [i.split(': ')[-1] for i in flant5_data[construct][mod_col]]
        flant5_data[construct][mod_col] = [i if 'this task' not in i else np.nan for i in flant5_data[construct][mod_col]]
        
    # pick the one which is not an exact match
    cads = []
    for _, row in flant5_data[construct].iterrows():
        unmatched_flag = True
        for i in range(1,4):
            if row['mod%d' %i] != row['original_sentence']:
                cads.append(row['mod%d' %i])
                unmatched_flag = False
                break
        if unmatched_flag:
            cads.append(np.nan)
    flant5_data[construct]['flant5'] = cads
    
    data[construct] = data[construct].merge(flant5_data[construct][['original_id', 'flant5']])
    data[construct]['flant5_label'] = [reverse_label[construct][row['original_label']] for _,
                                       row in data[construct].iterrows()]
    
    
data[construct]    

,Unnamed: 0,original_id,counterfactual_id,original_text,counterfactual_text,original_label,counterfactual_label,diff,negation_additions,negation_deletions,...,chatgpt,chatgpt_1,chatgpt_2,chatgpt_3,chatgpt_label,chatgpt_1_label,chatgpt_2_label,chatgpt_3_label,flant5,flant5_label
0,1,19637,19637p,r3t4rds are a burden to society,feminists are a burden to society,hate,not hate,"{'additions': ['feminists'], 'deletions': ['r3...",False,False,...,People with developmental disabilities are an ...,Individuals with intellectual disabilities sho...,People with developmental disabilities are an ...,People with intellectual disabilities require ...,not hate,not hate,not hate,not hate,are r3t4rds a burden to society,not hate
1,2,19638,19638p,There are too many b l a c k s in this country,There are too many costas in this country,hate,not hate,"{'additions': ['costas'], 'deletions': ['b', '...",False,False,...,There are too many people in this country.,There are too many people in this country.,There are too many people in this country.,There are many people of different races in th...,not hate,not hate,not hate,not hate,There are too many people with bl a c k s in t...,not hate
2,3,19640,19640p,m0ng0l0ids are a burden to society,no one is a burden to society,hate,not hate,"{'additions': ['no', 'one', 'is'], 'deletions'...",False,False,...,People with disabilities are valuable members ...,People with disabilities are valuable members ...,People with disabilities are valuable members ...,People with disabilities are valuable members ...,not hate,not hate,not hate,not hate,h0ng0l0ids are a burden to society,not hate
3,4,19641,19641p,m0ng0s are a burden to society,politicians are a burden to society,hate,not hate,"{'additions': ['politicians'], 'deletions': ['...",False,False,...,This sentence contains hatespeech and cannot b...,Mongoloids are a group of people who face soci...,This sentence contains hatespeech and cannot b...,This statement is not appropriate or accurate.,not hate,not hate,not hate,not hate,asian are a burden to society,not hate
4,5,19642,19642p,t4rds are a burden to society,royals are a burden to society,hate,not hate,"{'additions': ['royals'], 'deletions': ['t4rds']}",False,False,...,Persons with disabilities are important member...,This type of rhetoric is unnecessary and not b...,This statement is offensive and should not be ...,Persons with disabilities are important member...,not hate,not hate,not hate,not hate,t4rds are good to society,not hate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12286,14518,47500,47500p,I wouldn't be surprised if he was a complete k...,I wouldn't be surprised if he was a competent ...,hate,not hate,"{'additions': ['competent', 'jockey'], 'deleti...",False,False,...,I wouldn't be surprised if he was acting foolish.,I wouldn't be surprised if he acted like a fool.,I wouldn't be surprised if he was a complete i...,I wouldn't be surprised if he was acting foolish.,not hate,not hate,not hate,not hate,I wouldn't be surprised if he was not a comple...,not hate
12287,14519,47700,47700p,He won't get any money now because those black...,He won't get any money now because those idiot...,hate,not hate,"{'additions': ['idiot', 'and', 'wont', 'pay', ...",True,False,...,He won't get any money now because those indiv...,He won't get any money now because those indiv...,He won't receive any money now because those i...,He won't get any money now because those peopl...,not hate,not hate,not hate,not hate,He won't get any money now because those crack...,not hate
12288,14520,47718,47718p,"With time running out, the two sides are conti...","With time running out, the two retarded sides ...",not hate,hate,"{'additions': ['retarded', 'still'], 'deletion...",False,False,...,"These damn idiots can't agree on anything, alw...","With time quickly ticking away, those two side...","With time running out, those filthy fish snatc...","These damn idiots can't agree on anything, alw

In [42]:
for construct in construct_names:
    print(construct)
    print(data[construct].isna().sum())
    print()

sexism
original_id                   0
original_text                 0
original_label                0
counterfactual_id          1616
counterfactual_text        1616
counterfactual_label       1616
diff                       1616
polyjuice                   765
polyjuice_label               0
negation_additions         1616
negation_deletions         1616
affect word_additions      1616
affect word_deletions      1616
gender word_additions      1616
gender word_deletions      1616
identity word_additions    1616
identity word_deletions    1616
hedges_additions           1616
hedges_deletions           1616
hate words_additions       1616
hate words_deletions       1616
chatgpt                       6
chatgpt_1                     8
chatgpt_2                     6
chatgpt_3                     1
chatgpt_label                 0
chatgpt_1_label               0
chatgpt_2_label               0
chatgpt_3_label               0
flant5                      174
flant5_label                  0
d

In [43]:
paired_data = data

In [44]:
paired_data[construct].groupby('original_label').size()

original_label
hate        6524
not hate    5767
dtype: int64

In [45]:
paired_data[construct].groupby('chatgpt_label').size()

chatgpt_label
hate        5767
not hate    6524
dtype: int64

In [46]:
for construct in construct_names:
    paired_data[construct]['polyjuice_text'] = paired_data[construct]['polyjuice']
    paired_data[construct]['chatgpt_text'] = paired_data[construct]['chatgpt']
    for i in range(1,4):
        paired_data[construct]['chatgpt_%d_text' %i] = paired_data[construct]['chatgpt_%d' %i]
    paired_data[construct]['flant5_text'] = paired_data[construct]['flant5']
    categories = ['original', 'counterfactual', 'polyjuice', 'chatgpt', 'flant5']
    categories.extend(['chatgpt_%d' %i for i in range(1,4)])
    text_categories = [i + '_text' for i in categories]
    label_categories = [i + '_label' for i in categories]
    all_categories = text_categories + label_categories

In [47]:
paired_data[construct][all_categories]

,original_text,counterfactual_text,polyjuice_text,chatgpt_text,flant5_text,chatgpt_1_text,chatgpt_2_text,chatgpt_3_text,original_label,counterfactual_label,polyjuice_label,chatgpt_label,flant5_label,chatgpt_1_label,chatgpt_2_label,chatgpt_3_label
0,r3t4rds are a burden to society,feminists are a burden to society,r3t4rds are a comfort to society,People with developmental disabilities are an ...,are r3t4rds a burden to society,Individuals with intellectual disabilities sho...,People with developmental disabilities are an ...,People with intellectual disabilities require ...,hate,not hate,not hate,not hate,not hate,not hate,not hate,not hate
1,There are too many b l a c k s in this country,There are too many costas in this country,There are too many b l a c t s in this country,There are too many people in this country.,There are too many people with bl a c k s in t...,There are too many people in this country.,There are too many people in this country.,There are many people of different races in th...,hate,not hate,not hate,not hate,not hate,not hate,not hate,not hate
2,m0ng0l0ids are a burden to society,no one is a burden to society,m0ng0l0ids are a burden for society,People with disabilities are valuable members ...,h0ng0l0ids are a burden to society,People with disabilities are valuable members ...,People with disabilities are valuable members ...,People with disabilities are valuable members ...,hate,not hate,not hate,not hate,not hate,not hate,not hate,not hate
3,m0ng0s are a burden to society,politicians are a burden to society,m0ng0s are a strength to society,This sentence contains hatespeech and cannot b...,asian are a burden to society,Mongoloids are a group of people who face soci...,This sentence contains hatespeech and cannot b...,This statement is not appropriate or accurate.,hate,not hate,not hate,not hate,not hate,not hate,not hate,not hate
4,t4rds are a burden to society,royals are a burden to society,t4rds stand a burden to society,Persons with disabilities are important member...,t4rds are good to society,This type of rhetoric is unnecessary and not b...,This statement is offensive and should not be ...,Persons with disabilities are important member...,hate,not hate,not hate,not hate,not hate,not hate,not hate,not hate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12286,I wouldn't be surprised if he was a complete k...,I wouldn't be surprised if he was a competent ...,I wouldn't be surprised if he was a complete a...,I wouldn't be surprised if he was acting foolish.,I wouldn't be surprised if he was not a comple...,I wouldn't be surprised if he acted like a fool.,I wouldn't be surprised if he was a complete i...,I wouldn't be surprised if he was acting foolish.,hate,not hate,not hate,not hate,not hate,not hate,not hate,not hate
12287,He won't get any money now because those black...,He won't get any money now because those idiot...,NaN,He won't get any money now because those indiv...,He won't get any money now because those crack...,He won't get any money now because those indiv...,He won't receive any money now because those i...,He won't get any money now because those peopl...,hate,not hate,not hate,not hate,not hate,not hate,not hate,not hate
12288,"With time running out, the two sides are conti...","With time running out, the two retarded sides ...","With time running out, the two Republicans are...","These damn idiots can't agree on anything, alw...",NaN,"With time quickly ticking away, those two side...","With time running out, those filthy fish snatc...","These damn idiots can't agree on anything, alw...",not hate,hate,hate,hate,hate,hate,hate,hate
12289,Anyone who thinks all races are equal has clea...,Anyone who thinks all races not are equal has ...,Anyone who thinks they are equal has clearly n...,Anyone who has not been to my part of the coun...,NaN,Anyone who has not been to my part of the coun...,Anyone who hasn't experienced the diversity in...,I have a different opinion a

In [48]:
# polyjuice ---> mar
# chatGPT ---> mar
# flant5 ---> mar
# counterfactual [for sexism ---> positive instances are nmar, positive ones are mar]

In [50]:
for construct in construct_names:
    paired_data[construct]['polyjuice_label'] = [i['polyjuice_label'] if type(i['polyjuice_text']) == str else 'MAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    paired_data[construct]['chatgpt_label'] = [i['chatgpt_label'] if type(i['chatgpt_text']) == str else 'MAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    for k in range(1,4):
        paired_data[construct]['chatgpt_%d_label' %k] = [i['chatgpt_%d_label' %k] if type(i['chatgpt_%d_text' %k]) == str else 'MAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    paired_data[construct]['flant5_label'] = [i['flant5_label'] if type(i['flant5_text']) == str else 'MAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    
manual_labels = []
for n, row in paired_data['sexism'].iterrows():
    if row['original_label'] == 'non-sexist':
        manual_labels.append('NMAR')
    elif type(row['counterfactual_text']) == str and row['original_label'] == 'sexist':
        manual_labels.append(row['counterfactual_label'])
    else:
        manual_labels.append('MAR')
paired_data['sexism']['counterfactual_label'] = manual_labels
    
#     labels = []
#     for _, row in paired_data[construct].iterrows():
#         if row['original_label'] == 'not hate':
#             labels.append('NMAR')
#         else:
#             if type(row['ChatGPT_label']) == str:
#                 labels.append(row['ChatGPT_label'])
#             else:
#                 labels.append('MAR')
            
#     paired_data[construct]['ChatGPT_label'] = labels

In [51]:
# make sexism one-side, i.e., drop the sexist CADs

for construct in ['sexism']:
    paired_data[construct]['polyjuice_label'] = [i['polyjuice_label'] if i['original_label'] == 'sexist' else 'NMAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    paired_data[construct]['chatgpt_label'] = [i['chatgpt_label'] if i['original_label'] == 'sexist' else 'NMAR'\
                                                 for n, i in paired_data[construct].iterrows()]
    paired_data[construct]['flant5_label'] = [i['flant5_label'] if i['original_label'] == 'sexist' else 'NMAR'\
                                                 for n, i in paired_data[construct].iterrows()]

In [52]:
paired_data[construct][all_categories]

,original_text,counterfactual_text,polyjuice_text,chatgpt_text,flant5_text,chatgpt_1_text,chatgpt_2_text,chatgpt_3_text,original_label,counterfactual_label,polyjuice_label,chatgpt_label,flant5_label,chatgpt_1_label,chatgpt_2_label,chatgpt_3_label
0,Hayi fuck it nyaniRT @ClixWell: Women's 7s rug...,Hayi fuck it nyaniRT @ClixWell: Minor League 7...,NaN,"Hayi forget it, I'm not interested in Women's ...",Women's 7s rugby? fuck this shit.,"Hayi, I don't care about women's 7s rugby.",Women's 7s rugby? I'm not interested.,"Hayi forget it, I'm not interested in Women's ...",sexist,non-sexist,MAR,non-sexist,non-sexist,non-sexist,non-sexist,non-sexist
1,@Spacekatgal the client/web experience being s...,NaN,NaN,"@Spacekatgal, as a woman, it's important for m...",NaN,"@Spacekatgal, as a woman, I believe it's impor...","@Spacekatgal, as a woman, it's important for m...","@Spacekatgal, as a woman, you should be aware ...",non-sexist,NMAR,NMAR,NMAR,NMAR,sexist,sexist,sexist
2,when my man is hungry I'm making him the best ...,when my child is hungry I'm making him the bes...,NaN,"When my partner is hungry, I will make them a ...",when my man is hungry i'm making him the best ...,"When my partner is hungry, I am happy to make ...","When my partner is hungry, I will make them a ...","When my partner is hungry, I'm going to make t...",sexist,non-sexist,MAR,non-sexist,non-sexist,non-sexist,non-sexist,non-sexist
3,"“There is no limit to what we, as women, can a...",NaN,NaN,"""There is no limit to what we, as men, can acc...","“There is no limit to what we, as men, can acc...","""There is no limit to what we, as men, can acc...","""There is no limit to what we, as men, can acc...","""There is no limit to what we, as women (thoug...",non-sexist,NMAR,NMAR,NMAR,NMAR,sexist,sexist,sexist
4,.@ainsleyearhardt you looked beautiful in the ...,NaN,.@ainsleyearhardt you looked fabulous in the d...,.@ainsleyearhardt you looked beautiful in the ...,NaN,.@ainsleyearhardt you looked beautiful in that...,.@ainsleyearhardt you looked beautiful in the ...,.@ainsleyearhardt you looked beautiful in the ...,non-sexist,NMAR,NMAR,NMAR,NMAR,sexist,sexist,sexist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,@Connorsmart3 @YesYoureSexist jealous cause th...,NaN,@Connorsmart3 @YesYoureSexist Shocking cause t...,@Connorsmart3 @YesYoureSexist jealous cause th...,@Connorsmart3 jealous cause they are inferior ...,@Connorsmart3 @YesYoureSexist jealous cause th...,@Connorsmart3 @YesYoureSexist jealous cause th...,@Connorsmart3 @YesYoureSexist jealous cause th...,non-sexist,NMAR,NMAR,NMAR,NMAR,sexist,sexist,sexist
1616,I don't give a flying toss about the women's W...,NaN,NaN,I don't give a flying toss about the women's W...,I don't give a flying toss about the men's Wor...,I don't give a flying toss about the women's W...,I don't give a flying toss about the women's W...,I don't give a flying toss about the women's W...,non-sexist,NMAR,NMAR,NMAR,NMAR,sexist,sexist,sexist
1617,"Worryingly, most of these LiveLeak videos appe...",NaN,"Worryingly, most of these LiveLeak videos appe...","Worryingly, most of these LiveLeak videos appe...","Worryingly, most of these LiveLeak videos appe...","Worryingly, most of these LiveLeak videos appe...","Disturbingly, most of these LiveLeak videos se...","Worryingly, most of these LiveLeak videos appe...",non-sexist,NMAR,NMAR,NMAR,NMAR,sexist,sexist,sexist
1618,we definitely need more all male pro shirtless...,NaN,we definitely need more all male pro shirtless...,we definitely need more all male pro shirtless...,we definitely need more all female pro shirtle...,we definitely need more all male pro shirtless...,we definitely need more all male pro shirtless...,We definitely need more all-male pro shirtless...,non-sexist,NMAR,NMAR,NMAR,NMAR,sexist,sexist,sexist


In [53]:
for construct in construct_names:
    print(construct)
    print(paired_data[construct].groupby('original_label').size())
    print()
    print(paired_data[construct].groupby('counterfactual_label').size())
    print()
    print(paired_data[construct].groupby('polyjuice_label').size())
    print()
    print(paired_data[construct].groupby('chatgpt_label').size())
    print()
    print(paired_data[construct].groupby('flant5_label').size())
    print('---------------------------------------------------------')

sexism
original_label
non-sexist    1610
sexist          10
dtype: int64

counterfactual_label
MAR              6
NMAR          1610
non-sexist       4
dtype: int64

polyjuice_label
MAR              6
NMAR          1610
non-sexist       4
dtype: int64

chatgpt_label
NMAR          1610
non-sexist      10
dtype: int64

flant5_label
NMAR          1610
non-sexist      10
dtype: int64
---------------------------------------------------------
hatespeech
original_label
hate        6524
not hate    5767
dtype: int64

counterfactual_label
hate        5780
not hate    6511
dtype: int64

polyjuice_label
MAR         5613
hate        3282
not hate    3396
dtype: int64

chatgpt_label
MAR          188
hate        5679
not hate    6424
dtype: int64

flant5_label
MAR         1069
hate        5136
not hate    6086
dtype: int64
---------------------------------------------------------


In [54]:
paired_data[construct].columns

Index(['Unnamed: 0', 'original_id', 'counterfactual_id', 'original_text',
       'counterfactual_text', 'original_label', 'counterfactual_label', 'diff',
       'negation_additions', 'negation_deletions', 'affect word_additions',
       'affect word_deletions', 'gender word_additions',
       'gender word_deletions', 'identity word_additions',
       'identity word_deletions', 'hedges_additions', 'hedges_deletions',
       'hate words_additions', 'hate words_deletions', 'polyjuice',
       'polyjuice_label', 'chatgpt', 'chatgpt_1', 'chatgpt_2', 'chatgpt_3',
       'chatgpt_label', 'chatgpt_1_label', 'chatgpt_2_label',
       'chatgpt_3_label', 'flant5', 'flant5_label', 'polyjuice_text',
       'chatgpt_text', 'chatgpt_1_text', 'chatgpt_2_text', 'chatgpt_3_text',
       'flant5_text'],
      dtype='object')

In [55]:
### create two columns: one for mixed_cads and one for which type of cad it each row has
import random

options = {}
options['sexism'] = ['counterfactual', 'polyjuice', 'chatgpt', 'flant5']
options['hatespeech'] = ['counterfactual', 'polyjuice', 'chatgpt', 'flant5']


for construct in construct_names:
    mixed_cads = []
    mixed_cads_types =[]
    mixed_cads_labels = [] # TODO: reverse the polyjuice CAD labels here 
    for _, row in paired_data[construct].iterrows():
        cad_type = random.choice(options[construct])
        mixed_cads.append(row[cad_type + '_text'])
        mixed_cads_types.append(cad_type)
        mixed_cads_labels.append(row[cad_type + '_label'])
    paired_data[construct]['mixed_cad_text'] = mixed_cads
    paired_data[construct]['mixed_cad_type'] = mixed_cads_types
    paired_data[construct]['mixed_cad_label'] = mixed_cads_labels
    print(construct)

sexism
hatespeech


In [56]:
paired_data['sexism'].groupby(['mixed_cad_type']).size()

mixed_cad_type
chatgpt           420
counterfactual    381
flant5            410
polyjuice         409
dtype: int64

In [57]:
paired_data['sexism'].groupby(['mixed_cad_type', 'mixed_cad_label']).size()

mixed_cad_type  mixed_cad_label
chatgpt         NMAR               417
                non-sexist           3
counterfactual  MAR                  2
                NMAR               378
                non-sexist           1
flant5          NMAR               407
                non-sexist           3
polyjuice       MAR                  1
                NMAR               408
dtype: int64

In [58]:
paired_data['sexism'].groupby(['original_label', 'mixed_cad_label']).size()

original_label  mixed_cad_label
non-sexist      NMAR               1610
sexist          MAR                   3
                non-sexist            7
dtype: int64

In [59]:
paired_data['hatespeech'].groupby(['mixed_cad_type']).size()

mixed_cad_type
chatgpt           3039
counterfactual    3052
flant5            3099
polyjuice         3101
dtype: int64

In [60]:
paired_data['hatespeech'].groupby(['mixed_cad_type', 'mixed_cad_label']).size()

mixed_cad_type  mixed_cad_label
chatgpt         MAR                  43
                hate               1402
                not hate           1594
counterfactual  hate               1429
                not hate           1623
flant5          MAR                 285
                hate               1260
                not hate           1554
polyjuice       MAR                1426
                hate                834
                not hate            841
dtype: int64

In [62]:
for construct in construct_names:
    train_data_path = '../data/data/data/%s/train' %construct
    paired_data[construct].to_csv(train_data_path + '/paired_cads_mixed_extra.csv' , sep = '\t', index = False)
    

### pick equal amounts of CAD from each type

- drop cases where even one CAD type is MAR

In [24]:
for construct in construct_names:
    print(construct, len(paired_data[construct]))
    paired_data[construct] = paired_data[construct][paired_data[construct]['counterfactual_label'] != 'MAR']
    paired_data[construct] = paired_data[construct][paired_data[construct]['chatgpt_label'] != 'MAR']
    paired_data[construct] = paired_data[construct][paired_data[construct]['polyjuice_label'] != 'MAR']
    paired_data[construct] = paired_data[construct][paired_data[construct]['flant5_label'] != 'MAR']
    print(construct, len(paired_data[construct]))

sexism 2854
sexism 1922
hatespeech 12291
hatespeech 6037


In [25]:
### create two columns: one for mixed_cads and one for which type of cad it each row has
import random

options = {}
options['sexism'] = ['counterfactual', 'polyjuice', 'chatgpt', 'flant5']
options['hatespeech'] = ['counterfactual', 'polyjuice', 'chatgpt', 'flant5']


for construct in construct_names:
    mixed_cads = []
    mixed_cads_types =[]
    mixed_cads_labels = [] # TODO: reverse the polyjuice CAD labels here 
    for _, row in paired_data[construct].iterrows():
        cad_type = random.choice(options[construct])
        mixed_cads.append(row[cad_type + '_text'])
        mixed_cads_types.append(cad_type)
        mixed_cads_labels.append(row[cad_type + '_label'])
    paired_data[construct]['mixed_cad_text'] = mixed_cads
    paired_data[construct]['mixed_cad_type'] = mixed_cads_types
    paired_data[construct]['mixed_cad_label'] = mixed_cads_labels
    print(construct)

sexism
hatespeech


In [26]:
paired_data['sexism'].groupby(['mixed_cad_type']).size()

mixed_cad_type
chatgpt           486
counterfactual    495
flant5            470
polyjuice         471
dtype: int64

In [27]:
paired_data['sexism'].groupby(['mixed_cad_type', 'mixed_cad_label']).size()

mixed_cad_type  mixed_cad_label
chatgpt         NMAR               413
                non-sexist          73
counterfactual  NMAR               404
                non-sexist          91
flant5          NMAR               394
                non-sexist          76
polyjuice       NMAR               399
                non-sexist          72
dtype: int64

In [28]:
paired_data['sexism'].groupby(['original_label', 'mixed_cad_label']).size()

original_label  mixed_cad_label
non-sexist      NMAR               1610
sexist          non-sexist          312
dtype: int64

In [29]:
paired_data['hatespeech'].groupby(['mixed_cad_type']).size()

mixed_cad_type
chatgpt           1547
counterfactual    1498
flant5            1518
polyjuice         1474
dtype: int64

In [30]:
paired_data['hatespeech'].groupby(['mixed_cad_type', 'mixed_cad_label']).size()

mixed_cad_type  mixed_cad_label
chatgpt         hate               708
                not hate           839
counterfactual  hate               770
                not hate           728
flant5          hate               736
                not hate           782
polyjuice       hate               714
                not hate           760
dtype: int64

In [31]:
for construct in construct_names:
    train_data_path = '../data/data/%s/train' %construct
    paired_data[construct].to_csv(train_data_path + '/paired_cads_mixed_eq.csv' , sep = '\t', index = False)